In [1]:
import sys

import pandas as pd
import numpy as np
from pathlib import Path
import yaml
from typing import *

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Window
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval
import itertools

from pyspark.ml.feature import VectorAssembler, MinMaxScaler, BucketedRandomProjectionLSH
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array

In [2]:
n_cores = 30

spark = (SparkSession.builder
         .master(f"local[{n_cores}]")
         .appName("Saturation")
         .config("spark.sql.shuffle.partitions", "500")
         .config("spark.driver.memory", "64g")
         .config("spark.driver.maxResultSize", "8g")
         .getOrCreate())

23/11/24 14:51:52 WARN Utils: Your hostname, muninn resolves to a loopback address: 127.0.1.1; using 192.168.86.20 instead (on interface enp8s0)
23/11/24 14:51:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/24 14:51:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/24 14:51:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def read_config(path: Path) -> Dict:
    with path.open("r") as config_file:
        config = yaml.safe_load(config_file)
    return config


def read_configs(base_path: str, spark_session: SparkSession) -> pyspark.RDD:
    completed_filenames = list(Path(base_path).glob("*/completed.txt"))
    configs = map(lambda x: x.parent / "config.yaml", completed_filenames)
    configs = map(read_config, configs)
    return spark_session.sparkContext.parallelize(configs)


def create_configs_df(configs: pyspark.RDD) -> DataFrame:
    config_columns = [
        "simulation_id",
        "slope",
        "r_stat_multiplier",
        "effective_radius_multiplier",
        "min_rim_percentage"
    ]
    return configs.map(lambda x: {k: v for k, v in x.items() if k in config_columns}).toDF().cache()
    

def quantize_value(value: float, delta: float) -> float:
    return np.round(int(np.round(value / delta, decimals=5)) * delta, decimals=5)


def get_min_max_n_buckets(column: str, delta: float, df) -> Tuple[float, float, int]:
    min_max_df = df.select(F.min(column), F.max(column)).toPandas()
    
    min_val = quantize_value(min_max_df.iloc[0, 0], delta)
    max_val = quantize_value(min_max_df.iloc[0, 1], delta)
    n_buckets = int(np.round((max_val - min_val) / delta, decimals=5)) + 1

    return min_val, max_val, n_buckets

In [4]:
base_path = "/data/saturation/n_craters_stop_condition_20230918"

In [5]:
r_stat = 5
study_region_size = 4000 * 4000 / r_stat ** 2

configs_df = create_configs_df(read_configs(base_path, spark))
data = spark.read.parquet(f"{base_path}/*/statistics_*.parquet").coalesce(500)

# State c2c nn dist in terms of r_stat and log-scale
data = data.select("*",
                   F.log10(F.col("center_to_center_nearest_neighbor_distance_mean") / F.lit(r_stat)).alias("log_mean_c2c_nn_dist"),
                   F.log10(F.col("n_craters_in_study_region") / study_region_size).alias("log_intensity"),
                   F.log10("areal_density").alias("log_ad")
                  )

In [ ]:
# def get_confidence_interval(data: DataFrame,
#                             configs: DataFrame,
#                             target: str,
#                             observation: Dict[str, float],
#                             bandwidths: Dict[str, float],
#                             simulation_id: Optional[int] = None) -> pd.DataFrame:
#     data.createOrReplaceTempView("data")
#     configs.createOrReplaceTempView("configs")
    
#     simulation_id_clause = f"AND data.simulation_id <> {simulation_id}\n" if simulation_id else ""
#     bandwidth_clauses = ""
#     for var, bandwidth in bandwidths.items():
#         observation_value = observation[var]
#         bandwidth_clauses += f"AND {var} BETWEEN {observation_value - bandwidth} AND {observation_value + bandwidth}\n"
    
#     result = spark.sql(f"""
#     SELECT
#         mean({target}) as target_mean,
#         count({target}) as n_obs,
#         count(distinct data.simulation_id) as n_unique_sims,
#         approx_percentile({target}, array(0.025, 0.05, 0.10, 0.5, 0.90, 0.95, 0.975), 10000) as percentiles
#     FROM
#         data
#         INNER JOIN configs
#             ON data.simulation_id = configs.simulation_id
#     WHERE
#         1=1
#         {simulation_id_clause}
#         {bandwidth_clauses}
#     """).toPandas()

#     return result

In [ ]:
# def get_confidence_intervals(data: DataFrame,
#                              configs: DataFrame,
#                              target: str,
#                              observations: pd.DataFrame,
#                              bandwidths: pd.DataFrame,
#                              spark_session: SparkSession) -> pd.DataFrame:
#     data.createOrReplaceTempView("data")
#     configs.createOrReplaceTempView("configs")

#     observations_df = spark_session.createDataFrame(observations).cache()
#     observations_df.createOrReplaceTempView("observations")
#     bandwidths_df = spark_session.createDataFrame(bandwidths).cache()
#     bandwidths_df.createOrReplaceTempView("bandwidths")

#     # Trick it into caching and broadcasting
#     observations_df.count()
#     bandwidths_df.count()

#     simulation_id_present = "simulation_id" in observations.columns
    
#     group_by_clause = ",\n".join([f"o.{x}" for x in observations.columns] + [f"b.{x}" for x in bandwidths.columns])
#     if simulation_id_present:
#         simulation_id_clause = "AND d.simulation_id <> o.simulation_id\n"
#         target_select_clause = "o.target AS target,\n"
#         bandwidths_select_clause = "o.simulation_id AS simulation_id, o.crater_id AS crater_id,\n"
#     else:
#         simulation_id_clause = ""
#         target_select_clause = ""
#         bandwidths_select_clause = ""
       
#     observations_select_clause = ""
#     bandwidth_clauses = ""
#     for var in observations.columns:
#         observations_select_clause += f"o.{var} as observation_{var},\n"
#         if var in bandwidths.columns:
#             bandwidth_clauses += f"AND d.{var} BETWEEN (o.{var} - b.{var}) AND (o.{var} + b.{var})\n"

#     for var in bandwidths.columns:
#         bandwidths_select_clause += f"b.{var} as bandwith_{var},\n"

#     query = f"""
#     SELECT
#         {observations_select_clause}
#         {bandwidths_select_clause}
#         {target_select_clause}
#         mean({target}) as target_mean,
#         count({target}) as n_obs,
#         count(distinct d.simulation_id) as n_unique_sims,
#         approx_percentile({target}, array(0.025, 0.10, 0.90, 0.975), 1000) as percentiles
#     FROM
#         (
#             SELECT
#                 configs.slope,
#                 configs.r_stat_multiplier,
#                 configs.effective_radius_multiplier,
#                 configs.min_rim_percentage,
#                 data.*
#             FROM
#                 data
#                 INNER JOIN configs
#                     ON data.simulation_id = configs.simulation_id
#         ) AS d
#         INNER JOIN observations o
#         INNER JOIN bandwidths b
#     WHERE
#         1=1
#         {simulation_id_clause}
#         {bandwidth_clauses}
#     GROUP BY
#         {group_by_clause}
#     """
#     result = spark.sql(query)

#     return result.toPandas()

In [6]:
# Using HyperOpt to optimize
def get_confidence_intervals(data: DataFrame,
                             observations: DataFrame,
                             predictor_variables: List[str],
                             bandwidths: pd.DataFrame,
                             spark_session: SparkSession) -> pd.DataFrame:
    data.createOrReplaceTempView("data")
    
    bandwidths_df = spark_session.createDataFrame(bandwidths).cache()
    bandwidths_df.createOrReplaceTempView("bandwidths")

    # Trick it into caching and broadcasting
    bandwidths_df.count()

    simulation_id_present = "simulation_id" in observations.columns
    
    group_by_clause = ",\n ".join([f"o.{x}" for x in predictor_variables] + [f"b.{x}" for x in bandwidths.columns])

    if "target" in observations.columns:
        group_by_clause += "\n, o.target"
        
    if simulation_id_present:
        simulation_id_clause = "AND d.simulation_id <> o.simulation_id\n"
        target_select_clause = "o.target AS target,"
        bandwidths_select_clause = "o.simulation_id AS simulation_id, o.crater_id AS crater_id,\n"
        group_by_clause += "\n, o.simulation_id, o.crater_id"
    else:
        simulation_id_clause = ""
        target_select_clause = ""
        bandwidths_select_clause = ""
        

    observations_select_clause = ""
    bandwidth_clauses = "1=1 "
    for var in predictor_variables:
        observations_select_clause += f"o.{var} as observation_{var},\n"
        if var in bandwidths.columns:
            bandwidth_clauses += f"AND d.{var} BETWEEN (o.{var} - b.{var}) AND (o.{var} + b.{var})\n"

    for var in predictor_variables:
        bandwidths_select_clause += f"b.{var} as bandwidth_{var},\n"

    query = f"""
    SELECT
        {observations_select_clause}
        {bandwidths_select_clause}
        {target_select_clause}
        mean(d.target) as target_mean,
        count(d.target) as n_obs,
        count(distinct d.simulation_id) as n_unique_sims,
        approx_percentile(d.target, array(0.025, 0.975), 500) as percentiles
    FROM
        observations o
        INNER JOIN bandwidths b
        LEFT JOIN data d
            ON {bandwidth_clauses}
    WHERE
        1=1
        {simulation_id_clause}
    GROUP BY
        {group_by_clause}
    """
    result = spark.sql(query)

    return result.toPandas()


def setup_dataset(data: DataFrame,
                  configs: DataFrame,
                  predictor_variables: List[str],
                  target: str,
                  spark: SparkSession):
    data.createOrReplaceTempView("data")
    configs.createOrReplaceTempView("configs")
    
    # Join data and config
    data_and_config_select_clause = ",\n".join(["data.simulation_id", "crater_id"] + predictor_variables)
    data_and_config_select_clause += f",\n {target} as target"
    query = f"""
    SELECT
        {data_and_config_select_clause}
    FROM
        data
        INNER JOIN configs
            ON data.simulation_id = configs.simulation_id
    """
    data_and_configs = spark.sql(query)

    return data_and_configs


def setup_datasets_for_optimization(data: DataFrame,
                                    configs: DataFrame,
                                    predictor_variables: List[str],
                                    target: str,
                                    data_sample_fraction: float,
                                    test_sample_fraction: float,
                                    spark: SparkSession,
                                    cache_data: bool = True,
                                    cache_sample: bool = True):
    data_and_configs = setup_dataset(data, configs, predictor_variables, target, spark)

    if cache_data:
        data_sample = data_and_configs.sample(data_sample_fraction).cache()
        data_sample.createOrReplaceTempView("data")
        data_sample.count()
    else:
        data_sample = data_and_configs.sample(data_sample_fraction)
        data_sample.createOrReplaceTempView("data")

    if cache_sample:
        test_sample = data_and_configs.sample(test_sample_fraction).cache()
        test_sample.createOrReplaceTempView("observations")
        test_sample.count()
    else:
        test_sample = data_and_configs.sample(test_sample_fraction)
        test_sample.createOrReplaceTempView("observations")
    
    return data_sample, test_sample
    

def create_objective_function(data_sample: DataFrame,
                              observations: DataFrame,
                              predictor_variables: List[str],
                              spark: SparkSession):
    def objective_function(args):
        min_obs = 100
        n_too_many_obs = 500000
        
        bandwidths_data = [[args[f"bandwidth_{x}"] for x in predictor_variables]]
        bandwidths = pd.DataFrame(bandwidths_data, columns=predictor_variables)
        
        result = get_confidence_intervals(data_sample, observations, predictor_variables, bandwidths, spark)

        ci_high = result.percentiles.map(lambda x: x[-1])
        ci_low = result.percentiles.map(lambda x: x[0])
        orders_of_magnitude_rms = np.sqrt((np.log10(ci_high / ci_low) ** 2).mean())
        orders_of_magnitude = np.log10(ci_high / ci_low).mean()
        orders_of_magnitude_stdev = np.log10(ci_high / ci_low).std()
        mean_n_unique_sims = result.n_unique_sims.mean()
        mean_n_obs = result.n_obs.mean()
        
        percent_inside_ci = ((result.target >= ci_low) & (result.target <= ci_high)).mean()

        # Penalize if more or less than 5% are outside of the CI
        deviance_from_95 = np.abs(0.95 - percent_inside_ci)
        deviance_loss_multiplier = 1.0 if deviance_from_95 < 0.02 else 1 + deviance_from_95 * 10.0
        
        # Penalize for the fraction with too few observations
        too_few = result.n_obs[(result.n_obs < min_obs) | (result.n_obs.isna())]
        n_obs_loss_multiplier = 1 + too_few.mean() * 100 if too_few.shape[0] > 0 else 1.0

        # Penalize for the fraction with too many observations
        too_many = result.n_obs > n_too_many_obs
        n_too_many_obs_loss_multiplier = 1 + too_many.mean() * 10 if too_many.shape[0] > 0 else 1.0

        loss = orders_of_magnitude_rms * deviance_loss_multiplier * n_obs_loss_multiplier * n_too_many_obs_loss_multiplier

        print(f"{args}, {loss:.3f}, {orders_of_magnitude:.4f}, {orders_of_magnitude_stdev:.4f}, {orders_of_magnitude_rms:.4f}, {n_obs_loss_multiplier:.4f}, {n_too_many_obs_loss_multiplier:.4f}, {deviance_loss_multiplier:.4f}, {mean_n_unique_sims}, {mean_n_obs:.4f}, ")
        
        return {
            "loss": loss,
            "params": args,
            "orders_of_magnitude": orders_of_magnitude,
            "orders_of_magnitude_rms": orders_of_magnitude_rms,
            "n_obs_loss_multiplier": n_obs_loss_multiplier,
            "deviance_loss_multiplier": deviance_loss_multiplier,
            "mean_n_unique_sims": mean_n_unique_sims,
            "mean_n_obs": mean_n_obs,
            "status": STATUS_OK
        }
        
    return objective_function

In [7]:
def make_updates_and_report(data: pd.DataFrame) -> DataFrame:
    data["95_ci_ratio"] = data.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
    data["95_ci_low"] = data.percentiles.map(lambda x: x[0])
    data["95_ci_high"] = data.percentiles.map(lambda x: x[-1])
    
    data["80_ci_low"] = data.percentiles.map(lambda x: x[1])
    data["80_ci_high"] = data.percentiles.map(lambda x: x[-2])

    ci_95_hits = ((data.target >= data["95_ci_low"]) & (data.target <= data["95_ci_high"])).mean()
    ci_80_hits = ((data.target >= data["80_ci_low"]) & (data.target <= data["80_ci_high"])).mean()

    data["95_percentile_absolute_range"] = data.percentiles.apply(lambda x: x[-1]) - data.percentiles.apply(lambda x: x[0])
    data["95_percentile_ratio"] = data.percentiles.apply(lambda x: x[-1]) / data.percentiles.apply(lambda x: x[0])
    perc_ratio = np.mean(np.log10(data["95_percentile_ratio"]))

    mean_n_obs = data.n_obs.mean()
    n_obs_05 = data.n_obs.quantile(0.05)
    n_obs_25 = data.n_obs.quantile(0.25)
    n_obs_75 = data.n_obs.quantile(0.75)

    print(f"{ci_95_hits:.3f}, {ci_80_hits:.3f}, {perc_ratio:.3f}, {mean_n_obs:.3f}, {n_obs_05:.3f}, {n_obs_25:.3f}, {n_obs_75:.3f}")

In [8]:
# New hyperopt run

In [9]:
n_samples = 5000
n_obs_total = 2754157738
data_sample_fraction = 0.25
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = [
    "slope",
    "log_mean_c2c_nn_dist",
    "log_ad"
]

data_sample, test_sample = setup_datasets_for_optimization(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark, cache_data=False, cache_sample=True)

trials = Trials()
space = {
    # "bandwidth_slope": hp.uniform("bandwidth_slope", 0.1499, 0.15),
    "bandwidth_slope": hp.choice("bandwidth_slope", [0.1]),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwidth_log_mean_c2c_nn_dist", 0.0005, 0.02),
    "bandwidth_log_ad": hp.uniform("bandwidth_log_ad", 0.001, 0.15),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=100)

  0%|             | 0/100 [00:00<?, ?trial/s, best loss=?]

{'bandwidth_log_ad': 0.10783117230901547, 'bandwidth_log_mean_c2c_nn_dist': 0.010874825931929153, 'bandwidth_slope': 0.1}, 14.919, 1.3742, 0.3377, 1.4150, 1.0000, 10.5430, 1.0000, 583.7412772903489, 3144004.0396, 
  1%| | 1/100 [1:00:09<99:16:09, 3609.79s/trial, best loss

{'bandwidth_log_ad': 0.06615375780316522, 'bandwidth_log_mean_c2c_nn_dist': 0.013230994044638926, 'bandwidth_slope': 0.1}, 14.602, 1.3657, 0.3445, 1.4085, 1.0000, 10.3675, 1.0000, 529.9022648439094, 2994619.1871, 
  2%| | 2/100 [1:57:28<95:31:21, 3509.00s/trial, best loss

{'bandwidth_log_ad': 0.06360108789894536, 'bandwidth_log_mean_c2c_nn_dist': 0.002056107778387604, 'bandwidth_slope': 0.1}, 39308.072, 1.3625, 0.3512, 1.4071, 5826.0000, 4.7951, 1.0000, 448.19893899204243, 460388.5885, 
  3%| | 3/100 [2:38:10<81:25:25, 3021.91s/trial, best loss

{'bandwidth_log_ad': 0.0156654683446627, 'bandwidth_log_mean_c2c_nn_dist': 0.008741557197830166, 'bandwidth_slope': 0.1}, 40234.747, 1.3442, 0.3706, 1.3944, 5201.0000, 5.5481, 1.0000, 346.75943684962255, 692079.5134, 
  4%| | 4/100 [3:21:21<76:03:03, 2851.91s/trial, best loss

...

{'bandwidth_log_ad': 0.13647093526980336, 'bandwidth_log_mean_c2c_nn_dist': 0.006513609398309092, 'bandwidth_slope': 0.1}, 14.680, 1.3755, 0.3387, 1.4166, 1.0000, 10.3634, 1.0000, 594.7123036115079, 2076707.9208, 
 97%|▉| 97/100 [84:08:58<3:02:39, 3653.26s/trial, best los

{'bandwidth_log_ad': 0.12479438262642509, 'bandwidth_log_mean_c2c_nn_dist': 0.013368955257382648, 'bandwidth_slope': 0.1}, 15.109, 1.3763, 0.3358, 1.4166, 1.0000, 10.6654, 1.0000, 612.0740665170373, 4102669.4338, 
 98%|▉| 98/100 [85:16:56<2:06:01, 3780.64s/trial, best los

{'bandwidth_log_ad': 0.12948053557376377, 'bandwidth_log_mean_c2c_nn_dist': 0.007280090142058937, 'bandwidth_slope': 0.1}, 14.762, 1.3753, 0.3385, 1.4163, 1.0000, 10.4226, 1.0000, 591.4690879412365, 2274276.1947, 
 99%|▉| 99/100 [86:09:23<59:50, 3590.70s/trial, best loss:

{'bandwidth_log_ad': 0.08399743575303709, 'bandwidth_log_mean_c2c_nn_dist': 0.0047082212495189076, 'bandwidth_slope': 0.1}, 121777.152, 1.3686, 0.3446, 1.4113, 9201.0000, 9.3779, 1.0000, 514.0748826770047, 1216214.0837, 
100%|█| 100/100 [86:56:27<00:00, 3129.88s/trial, best loss


23/12/03 11:11:30 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-7d5212aa-ac3e-45b0-8ac4-d6d667929773. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-7d5212aa-ac3e-45b0-8ac4-d6d667929773
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:177)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:113)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:94)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1231)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:364)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach

In [ ]:
"""
{'bandwidth_log_ad': 0.09573505726230133, 'bandwidth_log_mean_c2c_nn_dist': 0.005269400944514953, 'bandwidth_slope': 0.1}, 13.831, 1.3711, 0.3422, 1.4131, 1.0000, 9.7880, 1.0000, 538.0983472760661, 1448353.3440, 

n_samples = 5000
n_obs_total = 2754157738
data_sample_fraction = 0.25
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = [
    "slope",
    "log_mean_c2c_nn_dist",
    "log_ad"
]

data_sample, test_sample = setup_datasets_for_optimization(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark, cache_data=False, cache_sample=True)

trials = Trials()
space = {
    # "bandwidth_slope": hp.uniform("bandwidth_slope", 0.1499, 0.15),
    "bandwidth_slope": hp.choice("bandwidth_slope", [0.1]),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwidth_log_mean_c2c_nn_dist", 0.0005, 0.02),
    "bandwidth_log_ad": hp.uniform("bandwidth_log_ad", 0.001, 0.15),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=100)
"""

In [ ]:
"""
{'bandwidth_log_ad': 0.03499639653600532, 'bandwidth_log_mean_c2c_nn_dist': 0.009940201998102223, 'bandwidth_slope': 0.1}, 1.396, 1.3507, 0.3539, 1.3962, 1.0000, 1.0000, 431.40763052208837, 615367.3594, 

n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.1
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = [
    "slope",
    "log_mean_c2c_nn_dist",
    "log_ad"
]

data_sample, test_sample = setup_datasets_for_optimization(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark, cache_data=True)

trials = Trials()
space = {
    # "bandwidth_slope": hp.uniform("bandwidth_slope", 0.1499, 0.15),
    "bandwidth_slope": hp.choice("bandwidth_slope", [0.1]),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwidth_log_mean_c2c_nn_dist", 0.0005, 0.01),
    "bandwidth_log_ad": hp.uniform("bandwidth_log_ad", 0.025, 0.2),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=100)
"""

In [ ]:
"""
{'bandwidth_log_ad': 0.05003817530737366, 'bandwidth_log_mean_c2c_nn_dist': 0.019032186174695872, 'bandwidth_slope': 0.15}, 1.405, 0.3343, 1.4049, 1.0000, 1.0000, 753.8094758064516    

n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.1
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = [
    "slope",
    "log_mean_c2c_nn_dist",
    "log_ad"
]

data_sample, test_sample = setup_datasets_for_optimization(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark, cache_data=True)

trials = Trials()
space = {
    # "bandwidth_slope": hp.uniform("bandwidth_slope", 0.1499, 0.15),
    "bandwidth_slope": hp.choice("bandwidth_slope", [0.15]),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwidth_log_mean_c2c_nn_dist", 0.005, 0.1),
    "bandwidth_log_ad": hp.uniform("bandwidth_log_ad", 0.05, 0.3),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=100)
"""

In [ ]:
"""
Best configuration with settings:
n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.05
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = [
    "slope",
    "log_mean_c2c_nn_dist",
    "log_ad"
]

data_sample, test_sample = setup_datasets_for_optimization(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

trials = Trials()
space = {
    "bandwidth_slope": hp.uniform("bandwidth_slope", 0.05, 0.2),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwidth_log_mean_c2c_nn_dist", 0.05, 0.15),
    "bandwidth_log_ad": hp.uniform("bandwidth_log_ad", 0.05, 0.3),
}

{'bandwidth_log_ad': 0.07543308379510501, 'bandwidth_log_mean_c2c_nn_dist': 0.07542596980056125, 'bandwidth_slope': 0.051337832245347034}, 1.426, 0.3178, 1.4260, 1.0000, 1.0000, 384.2230695900858   
"""

In [ ]:
# Pulled apart for debugging

# observations = test_sample

# min_obs = 300

# # args = {
# #     "bandwidth_slope": 0.1,
# #     "bandwidth_log_mean_c2c_nn_dist": 0.1,
# #     "bandwidth_log_ad": 0.1
# # }

# # bandwidths_data = [[args[f"bandwidth_{x}"] for x in predictor_variables]]
# # bandwidths = pd.DataFrame(bandwidths_data, columns=predictor_variables)

# # result = get_confidence_intervals(data_sample, observations, predictor_variables, bandwidths, spark)

# ci_high = result.percentiles.map(lambda x: x[-1])
# ci_low = result.percentiles.map(lambda x: x[0])
# orders_of_magnitude = np.log10(ci_high / ci_low).mean()
# orders_of_magnitude_stdev = np.log10(ci_high / ci_low).std()

# percent_inside_ci = ((result.target >= ci_low) & (result.target <= ci_high)).mean()

# # Penalize if more or less than 5% are outside of the CI
# deviance_from_95 = np.abs(0.95 - percent_inside_ci)
# deviance_loss_multiplier = 1.0 if deviance_from_95 < 0.02 else 1 + deviance_from_95 * 10.0

# # Penalize for the fraction with too few observations
# too_few = result.n_obs[result.n_obs < min_obs]
# n_obs_loss_multiplier = 1 + too_few.mean() * 100 if too_few.shape[0] > 0 else 1.0

# loss = orders_of_magnitude * deviance_loss_multiplier * n_obs_loss_multiplier

# print(f"{args}, {loss:.3f}, {orders_of_magnitude_stdev:.4f}")

# {
#     "loss": loss,
#     "orders_of_magnitude": orders_of_magnitude,
#     "n_obs_loss_multiplier": n_obs_loss_multiplier,
#     "deviance_loss_multiplier": deviance_loss_multiplier,
#     "status": STATUS_OK
# }

In [ ]:
result.n_obs[result.n_obs < 300].shape

In [ ]:
# Using only slope
n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.01
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = ["slope"]

data_sample, test_sample = setup_datasets(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

# 0.007, 0.011, 0.019, 1.247, 0.4383
# 0.055, 0.004, 0.009, 1.268, 0.4417  


trials = Trials()
space = {
    "bandwidth_slope": hp.uniform("bandwidth_slope", 0.001, 0.1),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=30)

In [ ]:
# Using only areal density
n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.01
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = ["areal_density"]

data_sample, test_sample = setup_datasets(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

# 0.007, 0.011, 0.019, 1.247, 0.4383
# 0.055, 0.004, 0.009, 1.268, 0.4417  


trials = Trials()
space = {
    "bandwidth_areal_density": hp.uniform("bandwidth_areal_density", 0.001, 0.05),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=30)

In [ ]:
# Using areal density
n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.01
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = ["slope", "areal_density"]

data_sample, test_sample = setup_datasets(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

# 0.007, 0.011, 0.019, 1.247, 0.4383
# 0.055, 0.004, 0.009, 1.268, 0.4417  


trials = Trials()
space = {
    "bandwidth_slope": hp.uniform("bandwidth_slope", 0.001, 0.1),
    "bandwidth_areal_density": hp.uniform("bandwidth_areal_density", 0.001, 0.05),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=30)

In [ ]:
# Using just c2c
n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.01
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = ["log_mean_c2c_nn_dist"]

data_sample, test_sample = setup_datasets(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

# 0.007, 0.011, 0.019, 1.247, 0.4383
# 0.055, 0.004, 0.009, 1.268, 0.4417  


trials = Trials()
space = {
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwith_log_mean_c2c_nn_dist", 0.000001, 0.05),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=30)

In [ ]:
# Using all three w/areal density
# best: {'bandwidth_areal_density': 0.01698628794548847, 'bandwidth_log_mean_c2c_nn_dist': 0.017780010374043788, 'bandwidth_slope': 0.006946936017484633}, 1.324, 0.3964   

n_samples = 1000
n_obs_total = 2754157738
data_sample_fraction = 0.01
test_sample_fraction = n_samples / n_obs_total
target = "n_craters_added_in_study_region"
predictor_variables = ["slope", "log_mean_c2c_nn_dist", "areal_density"]

data_sample, test_sample = setup_datasets(data, configs_df, predictor_variables, target, data_sample_fraction, test_sample_fraction, spark)

# 0.007, 0.011, 0.019, 1.247, 0.4383
# 0.055, 0.004, 0.009, 1.268, 0.4417  


trials = Trials()
space = {
    "bandwidth_slope": hp.uniform("bandwidth_slope", 0.001, 0.1),
    "bandwidth_log_mean_c2c_nn_dist": hp.uniform("bandwith_log_mean_c2c_nn_dist", 0.000001, 0.05),
    "bandwidth_areal_density": hp.uniform("bandwidth_areal_density", 0.001, 0.05),
}

best = fmin(create_objective_function(data_sample, test_sample, predictor_variables, spark),
            space=space,
            algo=tpe.suggest,
            max_evals=500)

In [ ]:
# DSP
data_sample_fraction = 0.1
n_steps = 5
target = "n_craters_added_in_study_region"

observation = {
    "slope": 2.66,
    "log_mean_c2c_nn_dist": 0.6685316230432883,
    "log_intensity": -3.71673468421674
    # "areal_density": 0.13
}

steps = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.1,
    "log_intensity": 1.0
    # "areal_density": 0.01
}

bandwidths = {
    "slope": 0.2,
    "log_mean_c2c_nn_dist": 0.1,
    "log_intensity": 1.0
    # "areal_density": 0.1
}

predictor_variables = list(observation.keys())

data_and_configs = setup_dataset(data, configs_df, predictor_variables, target, spark)
data_and_configs = data_and_configs.sample(data_sample_fraction)
data_and_configs.createOrReplaceTempView("data")

# Construct the observations grid
observations_df = pd.DataFrame(
    map(dict, itertools.product(*[[(k, observation[k] + x * v) for x in range(-n_steps, n_steps + 1)]  for k, v in observation.items()]))
)
observations_df = spark.createDataFrame(observations_df).cache()
observations_df.count()
observations_df.createOrReplaceTempView("observations")

bandwidths_df = pd.DataFrame([bandwidths])

result = get_confidence_intervals(data_and_configs, observations_df, predictor_variables, bandwidths_df, spark)
result

In [ ]:
# DEE
data_sample_fraction = 1.0
n_steps = 5
target = "n_craters_added_in_study_region"

observation = {
    "slope": 2.10,
    "log_mean_c2c_nn_dist": 0.8136,
    "log_intensity": -4.402922308048596
    # "areal_density": 0.13
}

steps = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.1,
    "log_intensity": 1.0
    # "areal_density": 0.01
}

bandwidths = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.1,
    "log_intensity": 1.0
    # "areal_density": 0.1
}

predictor_variables = list(observation.keys())

data_and_configs = setup_dataset(data, configs_df, predictor_variables, target, spark)
data_and_configs = data_and_configs.sample(data_sample_fraction)
data_and_configs.createOrReplaceTempView("data")

# Construct the observations grid
observations_df = pd.DataFrame(
    map(dict, itertools.product(*[[(k, observation[k] + x * v) for x in range(-n_steps, n_steps + 1)]  for k, v in observation.items()]))
)
observations_df = spark.createDataFrame(observations_df).cache()
observations_df.count()
observations_df.createOrReplaceTempView("observations")

bandwidths_df = pd.DataFrame([bandwidths])

result = get_confidence_intervals(data_and_configs, observations_df, predictor_variables, bandwidths_df, spark)
result

In [ ]:
result1 = result

In [ ]:
# DEE, AD
data_sample_fraction = 1.0
n_steps = 3
target = "n_craters_added_in_study_region"

observation = {
    "slope": 2.10,
    "log_mean_c2c_nn_dist": 0.8136,
    # "log_intensity": -4.402922308048596
    "areal_density": 0.06
}

steps = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.2,
    # "log_intensity": 1.0
    "areal_density": 0.05
}

bandwidths = {
    "slope": 0.25,
    "log_mean_c2c_nn_dist": 0.2,
    # "log_intensity": 1.0
    "areal_density": 0.05
}

predictor_variables = list(observation.keys())

data_and_configs = setup_dataset(data, configs_df, predictor_variables, target, spark)
data_and_configs = data_and_configs.sample(data_sample_fraction)
data_and_configs.createOrReplaceTempView("data")

# Construct the observations grid
observations_df = pd.DataFrame(
    map(dict, itertools.product(*[[(k, observation[k] + x * v) for x in range(-n_steps, n_steps + 1)]  for k, v in observation.items()]))
)
observations_df = spark.createDataFrame(observations_df).cache()
observations_df.count()
observations_df.createOrReplaceTempView("observations")

bandwidths_df = pd.DataFrame([bandwidths])

result = get_confidence_intervals(data_and_configs, observations_df, predictor_variables, bandwidths_df, spark)
result

In [ ]:
result.n_unique_sims.mean()

In [ ]:
# Searching for bandwidths in c2c_nn

In [ ]:
def get_nearest_neighbors(data: DataFrame,
                          observation: Dict[str, float],
                          predictor_variables: List[str],
                          scaling_factors: Dict[str, float],
                          n_neighbors: int) -> pd.DataFrame:
    assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in predictor_variables]
    scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in predictor_variables]
    pipeline = Pipeline(stages=assemblers + scalers)
    scaler = pipeline.fit(data_without_nulls)
    scaled = scaler.transform(data_without_nulls)

    assembler = VectorAssembler(inputCols=[f"{x}_scaled" for x in predictor_variables], outputCol="features")
    ready_to_model = assembler.transform(scaled_fixed)
    ready_to_model = ready_to_model.select(*[x for x in ready_to_model.columns if "scaled_vec" not in x])

    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", seed=12345, bucketLength=1.0)
    model = brp.fit(ready_to_model)

    vec = pd.DataFrame([[observation[x] for x in predictor_variables]], columns=predictor_variables)
    vec_scaled = assembler.transform(scaler.transform(spark.createDataFrame(vec)))

    return model.approxNearestNeighbors(ready_to_model, vec_scaled.select("features").collect()[0].features, n_neighbors).toPandas()

In [ ]:
data_sample_fraction = 1.0
n_steps = 3
target = "n_craters_added_in_study_region"

observation = {
    "slope": 2.10,
    "log_mean_c2c_nn_dist": 0.8136,
    # "log_intensity": -4.402922308048596
    "areal_density": 0.06
}

steps = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.2,
    # "log_intensity": 1.0
    "areal_density": 0.05
}

bandwidths = {
    "slope": 0.25,
    "log_mean_c2c_nn_dist": 0.2,
    # "log_intensity": 1.0
    "areal_density": 0.05
}

predictor_variables = list(observation.keys())
data_and_configs = setup_dataset(data, configs_df, predictor_variables, target, spark)

In [ ]:
# data_without_nulls = data_and_configs.sample(0.05).dropna().cache()
data_without_nulls = data_and_configs.dropna()

In [ ]:
scaling_factors = {
    "slope": 1,
    "log_mean_c2c_nn_dist": 1,
    "areal_density": 0.25
}
res1 = get_nearest_neighbors(data_without_nulls, observation, predictor_variables, scaling_factors, 1000)
res1

In [ ]:
scaling_factors = {
    "slope": 1.5,
    "log_mean_c2c_nn_dist": 1,
    "areal_density": 0.25
}
res2 = get_nearest_neighbors(data_without_nulls, observation, predictor_variables, scaling_factors, 1000)
res2

In [ ]:
scaling_factors = {
    "slope": 2,
    "log_mean_c2c_nn_dist": 1,
    "areal_density": 0.25
}
res3 = get_nearest_neighbors(data_without_nulls, observation, predictor_variables, scaling_factors, 100)
res3

In [ ]:
scaling_factors = {
    "slope": 1,
    "log_mean_c2c_nn_dist": 1,
    "areal_density": 0.1
}
res4 = get_nearest_neighbors(data_without_nulls, observation, predictor_variables, scaling_factors, 100)
res4

In [ ]:
plt.hist(res4.slope)

In [ ]:
nnd_filtered = data_without_nulls.where(
    F.col("log_mean_c2c_nn_dist").between(0.7, 0.9)
).cache()

In [ ]:
nnd_filtered.count()

In [ ]:
observation

In [ ]:
df = nnd_filtered.select(F.percentile_approx("areal_density", [0.025, 0.50, 0.975], 1000), F.percentile_approx("slope", [0.025, 0.50, 0.975], 1000)).toPandas()
df

In [ ]:
np.log10(0.12)

In [ ]:
# 1, DEE
d = data_without_nulls.withColumn("log_ad", F.log10("areal_density"))

observation = {
    "slope": 2.10,
    "log_mean_c2c_nn_dist": 0.8136,
    # "log_intensity": -4.402922308048596
    "log_ad": np.log10(0.06)
}

bandwidths = {
    "slope": 0.2,
    "log_mean_c2c_nn_dist": 0.1,
    # "log_intensity": -4.402922308048596
    "log_ad": 0.15
}

res = d
for k, v in observation.items():
    res = res.where(F.col(k).between(v - bandwidths[k], v + bandwidths[k]))

res_df = res.toPandas()
dee_df = res_df

In [ ]:
n_orders_mag = np.log10(res_df.target.quantile(0.975) / res_df.target.quantile(0.025))
n_orders_mag

In [ ]:
# 2, DSP
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685

observation = {
    "slope": 2.66,
    "log_mean_c2c_nn_dist": 0.6685,
    "log_ad": np.log10(0.13)
}

bandwidths = {
    "slope": 0.2,
    "log_mean_c2c_nn_dist": 0.1,
    "log_ad": 0.15
}

res = d
for k, v in observation.items():
    res = res.where(F.col(k).between(v - bandwidths[k], v + bandwidths[k]))

res_df = res.toPandas()
dsp_df = res_df

In [ ]:
n_orders_mag = np.log10(res_df.target.quantile(0.975) / res_df.target.quantile(0.025))
n_orders_mag

In [ ]:
# 3, DICP
observation = {
    "slope": 2.08,
    "log_mean_c2c_nn_dist": 0.9338405903389017,
    # "log_intensity": -4.402922308048596
    "log_ad": np.log10(0.31)
}

bandwidths = {
    "slope": 0.2,
    "log_mean_c2c_nn_dist": 0.1,
    # "log_intensity": -4.402922308048596
    "log_ad": 0.15
}

res = d
for k, v in observation.items():
    res = res.where(F.col(k).between(v - bandwidths[k], v + bandwidths[k]))

res_df = res.toPandas()
dicp_df = res_df

In [ ]:
n_orders_mag = np.log10(res_df.target.quantile(0.975) / res_df.target.quantile(0.025))
n_orders_mag

In [ ]:
# 4, DDCP
d = data_without_nulls.withColumn("log_ad", F.log10("areal_density"))

observation = {
    "slope": 2.15,
    "log_mean_c2c_nn_dist": 0.6685,
    "log_ad": np.log10(0.35)
}

bandwidths = {
    "slope": 0.2,
    "log_mean_c2c_nn_dist": 0.1,
    "log_ad": 0.15
}

res = d
for k, v in observation.items():
    res = res.where(F.col(k).between(v - bandwidths[k], v + bandwidths[k]))

In [ ]:
percentiles = res.select(F.percentile_approx("target", [0.025, 0.975], 1000)).toPandas()

In [ ]:
np.log10(percentiles.iloc[0, 0][1] / percentiles.iloc[0, 0][0])

In [ ]:
n_orders_mag = np.log10(res_df.target.quantile(0.975) / res_df.target.quantile(0.025))
n_orders_mag

In [ ]:
columns_to_scale = ["slope", "log_mean_c2c_nn_dist", "areal_density"]

assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scaler = pipeline.fit(data_without_nulls)
scaled = scaler.transform(data_without_nulls)

In [ ]:
scaling_factors = {
    "slope": 0.25,
    "log_mean_c2c_nn_dist": 1,
    "areal_density": 0.25
}
scaled_fixed = scaled.select(
    *data_without_nulls.columns,
    *[(vector_to_array(F.col(f"{x}_scaled")).getItem(0) * F.lit(scaling_factors[x])).alias(f"{x}_scaled") for x in columns_to_scale]
)

assembler = VectorAssembler(inputCols=[f"{x}_scaled" for x in columns_to_scale], outputCol="features")
ready_to_model = assembler.transform(scaled_fixed)
ready_to_model = ready_to_model.select(*[x for x in ready_to_model.columns if "scaled_vec" not in x])

In [ ]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", seed=12345, bucketLength=1.0)
model = brp.fit(ready_to_model)

In [ ]:
vec = pd.DataFrame([[2.10, 0.8136, 0.06]], columns=columns_to_scale)
vec_scaled = assembler.transform(scaler.transform(spark.createDataFrame(vec)))

In [ ]:
res = model.approxNearestNeighbors(ready_to_model, vec_scaled.select("features").collect()[0].features, 50).toPandas()

In [ ]:
res

In [ ]:
res2 = model.approxNearestNeighbors(ready_to_model, Vectors.dense 50).collect()

In [ ]:
assembler = VectorAssembler(outputCol="features")

assemblers = 
assembler.setInputCols(["slope", "log_mean_c2c_nn_dist", "areal_density"])

scaler = MinMaxScaler(outputCol="scaled_features")
scaler.setInputCol("features")
scaler.fit(assembler.transform(data_and_configs))

In [ ]:
assembler.transform(data_and_configs)

In [ ]:
df = data_and_configs.where((F.col("areal_density") < 0.08) & (F.col("log_mean_c2c_nn_dist").between(0.7, 0.8))).toPandas()

In [ ]:
df.slope.describe()

In [ ]:
total: 2.5b
ad limit: 23,500,945
ad + log_c2c: 1,056,227

In [ ]:
data.select(F.percentile_approx("log_intensity", [0.05, 0.25, 0.5, 0.75, 0.95], 1000), F.mean("log_intensity"), F.median("log_intensity"), F.stddev_samp("log_intensity")).toPandas()

In [ ]:
data_sample_fraction = 0.01
n_steps = 5
target = "n_craters_added_in_study_region"
predictor_variables = ["slope", "log_mean_c2c_nn_dist", "areal_density"]

# data_and_configs = setup_datasets(data, configs_df, predictor_variables, target, spark)
# data_and_configs = data_and_configs.sample(data_sample_fraction)

observation = {
    "slope": 2.10,
    "target_log_mean_c2c_nn_dist": 0.8136322010294104,
    "areal_density": 0.13
}

steps = {
    "slope": 0.05,
    "target_log_mean_c2c_nn_dist": 0.05,
    "areal_density": 0.01
}





In [ ]:
steps_df

In [ ]:
pd.DataFrame([dict(x) for x in steps])

In [ ]:
# DSP
target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
    [target_slope, target_log_mean_c2c_nn_dist],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.001],
    [0.01, 0.005],
    [0.005, 0.001],
    [0.005, 0.005],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result

In [ ]:
# DEE
target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
    [target_slope, target_log_mean_c2c_nn_dist],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.0005],
    [0.01, 0.001],
    [0.01, 0.005],
    [0.025, 0.0005],
    [0.025, 0.001],
    [0.025, 0.005],
    [0.05, 0.0005],
    [0.05, 0.001],
    [0.05, 0.005],
    [0.1, 0.0001],
    [0.1, 0.00025],
    [0.1, 0.0005],
    [0.1, 0.001],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result.sort_values([f"observation_{x}" for x in observations.columns])

In [ ]:
# DEE

target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
    [target_slope, target_log_mean_c2c_nn_dist],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.0005],
    [0.01, 0.001],
    [0.01, 0.005],
    [0.025, 0.0005],
    [0.025, 0.001],
    [0.025, 0.005],
    [0.05, 0.0005],
    [0.05, 0.001],
    [0.05, 0.005],
    [0.1, 0.0001],
    [0.1, 0.00025],
    [0.1, 0.0005],
    [0.1, 0.001],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result.sort_values([f"observation_{x}" for x in observations.columns])

In [ ]:
# Dione 3 (DICP)
target = "n_craters_added_in_study_region"
target_slope = 2.08
slope_ci_width = 0.09
target_log_mean_c2c_nn_dist = 0.9338405903389017

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
    [target_slope, target_log_mean_c2c_nn_dist],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.0005],
    [0.01, 0.001],
    [0.01, 0.005],
    [0.025, 0.0005],
    [0.025, 0.001],
    [0.025, 0.005],
    [0.05, 0.0005],
    [0.05, 0.001],
    [0.05, 0.005],
    [0.1, 0.0001],
    [0.1, 0.00025],
    [0.1, 0.0005],
    [0.1, 0.001],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result.sort_values([f"observation_{x}" for x in observations.columns])

In [ ]:
# Dione 4 (DDCP)
target = "n_craters_added_in_study_region"
target_slope = 2.15
slope_ci_width = 0.05
target_log_mean_c2c_nn_dist = 0.5480628421533491

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
    [target_slope, target_log_mean_c2c_nn_dist],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.0005],
    [0.01, 0.001],
    [0.01, 0.005],
    [0.025, 0.0005],
    [0.025, 0.001],
    [0.025, 0.005],
    [0.05, 0.0005],
    [0.05, 0.001],
    [0.05, 0.005],
    [0.1, 0.0001],
    [0.1, 0.00025],
    [0.1, 0.0005],
    [0.1, 0.001],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result.sort_values([f"observation_{x}" for x in observations.columns])

In [ ]:
# The Grid
target = "n_craters_added_in_study_region"

log_mean_c2c_nn_dist_delta = 0.05
slope_delta = 0.05

min_slope = configs_df.select(F.min("slope")).toPandas().iloc[0,0]
max_slope = configs_df.select(F.max("slope")).toPandas().iloc[0,0]

min_log_mean_c2c_nn_dist = data.select(F.min("log_mean_c2c_nn_dist")).toPandas().iloc[0,0]
max_log_mean_c2c_nn_dist = data.select(F.max("log_mean_c2c_nn_dist")).toPandas().iloc[0,0]

min_log_mean_c2c_nn_dist = quantize_value(min_log_mean_c2c_nn_dist + log_mean_c2c_nn_dist_delta * 2, log_mean_c2c_nn_dist_delta)
max_log_mean_c2c_nn_dist = quantize_value(max_log_mean_c2c_nn_dist - log_mean_c2c_nn_dist_delta * 2, log_mean_c2c_nn_dist_delta)

log_c2c_nn_dist_choices = [round(min_log_mean_c2c_nn_dist + x * log_mean_c2c_nn_dist_delta, 4) for x in range(int((max_log_mean_c2c_nn_dist - min_log_mean_c2c_nn_dist) / log_mean_c2c_nn_dist_delta))]

min_slope = quantize_value(min_slope + 2 * slope_delta, slope_delta)
max_slope = quantize_value(max_slope - 2 * slope_delta, slope_delta)
slope_choices = [round(min_slope + x * slope_delta, 4) for x in range(int((max_slope - min_slope) / slope_delta))]

observations_data = [
    [x, y]
    for x in slope_choices
    for y in log_c2c_nn_dist_choices
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [0.01, 0.0005],
    [0.01, 0.001],
    [0.01, 0.005],
    [0.025, 0.0005],
    [0.025, 0.001],
    [0.025, 0.005],
    [0.05, 0.0005],
    [0.05, 0.001],
    [0.05, 0.005],
    [0.1, 0.0001],
    [0.1, 0.00025],
    [0.1, 0.0005],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result.sort_values([f"observation_{x}" for x in observations.columns])

In [ ]:
result.to_parquet("quantile_kernel_regression_grid_multiple_bandwidths.parquet")

In [ ]:
result = result.sort_values([f"bandwith_{x}" for x in bandwidths.columns])

In [ ]:
# Dione region 1, DEE
target_slope = 2.10
target_log_mean_c2c_nn_dist = 0.8136322010294104
r = result[(result.observation_slope == 2.1) & (result.observation_log_mean_c2c_nn_dist == 0.80)].copy()
r["95_percentile_absolute_range"] = r.percentiles.apply(lambda x: x[-1]) - r.percentiles.apply(lambda x: x[0])
r["95_percentile_ratio"] = r.percentiles.apply(lambda x: x[-1]) / r.percentiles.apply(lambda x: x[0])
r

In [ ]:
# Dione region 2, DSP
target_slope = 2.66
target_log_mean_c2c_nn_dist = 0.6685316230432883
r = result[(result.observation_slope == 2.65) & (result.observation_log_mean_c2c_nn_dist == 0.65)].copy()
r["95_percentile_absolute_range"] = r.percentiles.apply(lambda x: x[-1]) - r.percentiles.apply(lambda x: x[0])
r["95_percentile_ratio"] = r.percentiles.apply(lambda x: x[-1]) / r.percentiles.apply(lambda x: x[0])
r

In [ ]:
# Dione region 3, DICP
target_slope = 2.08
target_log_mean_c2c_nn_dist = 0.9338405903389017
r = result[(result.observation_slope == 2.05) & (result.observation_log_mean_c2c_nn_dist == 0.95)].copy()
r["95_percentile_absolute_range"] = r.percentiles.apply(lambda x: x[-1]) - r.percentiles.apply(lambda x: x[0])
r["95_percentile_ratio"] = r.percentiles.apply(lambda x: x[-1]) / r.percentiles.apply(lambda x: x[0])
r

In [ ]:
# DICP with lower nn dist:
r = result[(result.observation_slope == 2.05) & (result.observation_log_mean_c2c_nn_dist == 0.90)].copy()
r["95_percentile_absolute_range"] = r.percentiles.apply(lambda x: x[-1]) - r.percentiles.apply(lambda x: x[0])
r["95_percentile_ratio"] = r.percentiles.apply(lambda x: x[-1]) / r.percentiles.apply(lambda x: x[0])
r

In [ ]:
# Dione 4 (DDCP)
target_slope = 2.15
target_log_mean_c2c_nn_dist = 0.5480628421533491
r = result[(result.observation_slope == 2.15) & (result.observation_log_mean_c2c_nn_dist == 0.55)].copy()
r["95_percentile_absolute_range"] = r.percentiles.apply(lambda x: x[-1]) - r.percentiles.apply(lambda x: x[0])
r["95_percentile_ratio"] = r.percentiles.apply(lambda x: x[-1]) / r.percentiles.apply(lambda x: x[0])
r

In [ ]:
# Dione region 1: DEE - extended
target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
slope_step = 0.05
target_log_mean_c2c_nn_dist = 0.8136
target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
dee_extended = result

In [ ]:
result.sort_values("95_ci_ratio")[:25]

In [ ]:
# Dione region 1: DEE
target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136

# observations_data = [
#     [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
#     [target_slope, target_log_mean_c2c_nn_dist],
#     [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
# ]
# observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

# bandwidths_data = [
#     [slope_ci_width, 0.005],
# ]
# bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

# result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
# result[result.observation_slope == target_slope]
target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
dee = result

In [ ]:
percentiles = result[result.observation_slope == target_slope].percentiles.iloc[0]
print(f"Orders of magnitude: {np.log10(percentiles[-1] / percentiles[0])}")
print(f"95% CI: ({percentiles[0]}, {percentiles[-1]})")

In [ ]:
dee.sort_values("95_ci_ratio")

In [ ]:
dee[(dee.observation_slope == 2.10) & (dee.observation_log_mean_c2c_nn_dist == 0.8136)]

In [ ]:
# Dione region 2: DSP
target = "n_craters_added_in_study_region"
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685

# observations_data = [
#     [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
#     [target_slope, target_log_mean_c2c_nn_dist],
#     [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
# ]
# observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

# bandwidths_data = [
#     [slope_ci_width, 0.005],
# ]
# bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

# result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
# result[result.observation_slope == target_slope]

target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
dsp_extended = result

In [ ]:
percentiles = result[result.observation_slope == target_slope].percentiles.iloc[0]
print(f"Orders of magnitude: {np.log10(percentiles[-1] / percentiles[0])}")
print(f"95% CI: ({percentiles[0]}, {percentiles[-1]})")

In [ ]:
dsp_extended.sort_values("95_ci_ratio")

In [ ]:
# Dione region 3: DICP
target = "n_craters_added_in_study_region"
target_slope = 2.08
slope_ci_width = 0.09
target_log_mean_c2c_nn_dist = 0.9338

# observations_data = [
#     [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
#     [target_slope, target_log_mean_c2c_nn_dist],
#     [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
# ]
# observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

# bandwidths_data = [
#     [slope_ci_width, 0.005],
# ]
# bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

# result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
# result[result.observation_slope == target_slope]

target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
dicp = result

In [ ]:
percentiles = result[result.observation_slope == target_slope].percentiles.iloc[0]
print(f"Orders of magnitude: {np.log10(percentiles[-1] / percentiles[0])}")
print(f"95% CI: ({percentiles[0]}, {percentiles[-1]})")

In [ ]:
dicp.sort_values("95_ci_ratio")

In [ ]:
# Dione region 3: DICP, D > 4000m
target = "n_craters_added_in_study_region"
target_slope = 2.08
slope_ci_width = 0.09
target_log_mean_c2c_nn_dist = 0.6304

# observations_data = [
#     [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
#     [target_slope, target_log_mean_c2c_nn_dist],
#     [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
# ]
# observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

# bandwidths_data = [
#     [slope_ci_width, 0.005],
# ]
# bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

# result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
# result[result.observation_slope == target_slope]

target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
dicp_4k_plus = result

In [ ]:
percentiles = result[result.observation_slope == target_slope].percentiles.iloc[0]
print(f"Orders of magnitude: {np.log10(percentiles[-1] / percentiles[0])}")
print(f"95% CI: ({percentiles[0]}, {percentiles[-1]})")

In [ ]:
dicp_4k_plus.sort_values("95_ci_ratio")

In [ ]:
# Dione region 4: DDCP
target = "n_craters_added_in_study_region"
target_slope = 2.15
slope_ci_width = 0.05
target_log_mean_c2c_nn_dist = 0.5481

# observations_data = [
#     [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist],
#     [target_slope, target_log_mean_c2c_nn_dist],
#     [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist],
# ]
# observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

# bandwidths_data = [
#     [slope_ci_width, 0.005],
# ]
# bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

# result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
# result[result.observation_slope == target_slope]
target_log_mean_c2c_nn_dist_step = 0.05


observations_data = [
    [target_slope + x * slope_step, target_log_mean_c2c_nn_dist + y * target_log_mean_c2c_nn_dist_step]
    for x in range(-5, 6)
    for y in range(-5, 6)
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist"])

bandwidths_data = [
    [slope_ci_width / 2, 0.0025],
    [slope_ci_width / 2, 0.005],
    [slope_ci_width, 0.0025],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result["95_ci_ratio"] = result.percentiles.map(lambda x: np.log10(x[-1] / x[0]))
result["95_ci_range"] = result.percentiles.map(lambda x: [x[0], x[1]])
ddcp = result

In [ ]:
percentiles = result[result.observation_slope == target_slope].percentiles.iloc[0]
print(f"Orders of magnitude: {np.log10(percentiles[-1] / percentiles[0])}")
print(f"95% CI: ({percentiles[0]}, {percentiles[-1]})")

In [ ]:
ddcp.sort_values("95_ci_ratio")

In [ ]:
# DEE w/AD

target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104
target_ad = 0.06

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist, target_ad],
    [target_slope, target_log_mean_c2c_nn_dist, target_ad],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist, target_ad],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist", "areal_density"])

bandwidths_data = [
    [0.01, 0.001, 0.1],
    [0.01, 0.005, 0.1],
    [0.005, 0.001, 0.1],
    [0.005, 0.005, 0.1],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist", "areal_density"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result

In [ ]:
# DEE w/Za

target = "n_craters_added_in_study_region"
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104
target_za = -0.23

observations_data = [
    [target_slope - slope_ci_width, target_log_mean_c2c_nn_dist, target_za],
    [target_slope, target_log_mean_c2c_nn_dist, target_za],
    [target_slope + slope_ci_width, target_log_mean_c2c_nn_dist, target_za],
]
observations = pd.DataFrame(observations_data, columns=["slope", "log_mean_c2c_nn_dist", "za"])

bandwidths_data = [
    [0.01, 0.001, 3],
    [0.01, 0.005, 3],
    [0.005, 0.001, 3],
    [0.005, 0.005, 3],
]
bandwidths = pd.DataFrame(bandwidths_data, columns=["slope", "log_mean_c2c_nn_dist", "za"])

result = get_confidence_intervals(data, configs_df, target, observations, bandwidths, spark)
result

In [ ]:
target = "n_craters_added_in_study_region"

observation = {
    "slope": 1.5,
    "log_mean_c2c_nn_dist": 1.8
}
bandwidths = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.05
}

result = get_confidence_interval(data, configs_df, target, observation, bandwidths)

In [ ]:
result

In [ ]:
simulation_id = None
target = "n_craters_added_in_study_region"

bandwidths = {
    "slope": 0.01,
    "log_mean_c2c_nn_dist": 0.001
}

In [ ]:
# Dione 1 (DEE)
target_slope = 2.10
slope_ci_width = 0.08
target_log_mean_c2c_nn_dist = 0.8136322010294104

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione1_result = result
result

In [ ]:
# Dione 2 (DSP)
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685316230432883
target_effective_radius_multiplier = 1.3

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione2_result = result
result

In [ ]:
# Dione 2 (DSP)
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685316230432883
target_effective_radius_multiplier = 1.3

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione2_result = result
result

In [ ]:
# Dione 2 (DSP)
# Wider slope bandwidth
target_slope = 2.8
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685316230432883
target_effective_radius_multiplier = 1.3

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione2_result = result
result

In [ ]:
# Dione 3 (DICP)
target_slope = 2.08
slope_ci_width = 0.09
target_log_mean_c2c_nn_dist = 0.9338405903389017

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione3_result = result
result

In [ ]:
# Dione 4 (DDCP)
target_slope = 2.15
slope_ci_width = 0.05
target_log_mean_c2c_nn_dist = 0.5480628421533491

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist
}
result = get_confidence_interval(data, configs_df, target, observation, bandwidths)
dione4_result = result
result

In [ ]:
# Dione 2 (DSP)
# With eff radius multiplier
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685316230432883
target_effective_radius_multiplier = 1.3

observation = {
    "slope": target_slope,
    "log_mean_c2c_nn_dist": target_log_mean_c2c_nn_dist,
    "effective_radius_multiplier": target_effective_radius_multiplier
}
bandwidths_with_eff = bandwidths.copy()
bandwidths_with_eff.update({"effective_radius_multiplier": 0.05})
result = get_confidence_interval(data, configs_df, target, observation, bandwidths_with_eff)
dione2_with_eff_result = result
result

In [ ]:
# Try using SQL
simulation_id = None
target = "n_craters_added_in_study_region"
bandwidths = {
    "slope": 0.1,
    "log_mean_c2c_nn_dist": 0.05
}

observation = {
    "slope": 1.5,
    "log_mean_c2c_nn_dist": 1.8
}


data.createOrReplaceTempView("data")
configs_df.createOrReplaceTempView("configs")

simulation_id_clause = f"AND data.simulation_id <> {simulation_id}\n" if simulation_id else ""
bandwidth_clauses = ""
for var, bandwidth in bandwidths.items():
    observation_value = observation[var]
    bandwidth_clauses += f"AND {var} BETWEEN {observation_value - bandwidth} AND {observation_value + bandwidth}\n"

result = spark.sql(f"""
SELECT
    *
    -- approx_percentile({target}, array(0.025, 0.5, 0.975), 10000) as percentiles
FROM
    data
    INNER JOIN configs
        ON data.simulation_id = configs.simulation_id
WHERE
    1=1
    {simulation_id_clause}
    {bandwidth_clauses}
""").toPandas()

In [ ]:
data.columns

In [ ]:
slope_delta = 0.1
log_mean_c2c_nn_dist_delta = 0.1
effective_radius_multiplier_delta = 0.1

min_slope, max_slope, n_slope_buckets = get_min_max_n_buckets("slope", slope_delta, configs_df)
min_log_mean_c2c_nn_dist, max_log_mean_c2c_nn_dist, n_log_mean_c2c_nn_dist_buckets = get_min_max_n_buckets("log_mean_c2c_nn_dist", log_mean_c2c_nn_dist_delta, data)
min_effective_radius_multiplier, max_effective_radius_multiplier, n_effective_radius_multiplier_buckets = get_min_max_n_buckets("effective_radius_multiplier", effective_radius_multiplier_delta, configs_df)

In [ ]:
targets_array = [
    [np.round(x, decimals=5), np.round(y, decimals=5), np.round(z, decimals=5)]
    for x in np.linspace(min_slope, max_slope, n_slope_buckets)
    for y in np.linspace(min_log_mean_c2c_nn_dist, max_log_mean_c2c_nn_dist, n_log_mean_c2c_nn_dist_buckets)
    for z in np.linspace(min_effective_radius_multiplier, max_effective_radius_multiplier, n_effective_radius_multiplier_buckets)
]

targets = pd.DataFrame(targets_array, columns=["target_slope", "target_log_mean_c2c_nn_dist", "target_effective_radius_multiplier"])
targets_spark_df = spark.createDataFrame(targets)
data_with_targets = data.join(F.broadcast(targets_spark_df), how="cross")

In [ ]:
data_with_configs = data_with_targets.join(F.broadcast(configs_df), on="simulation_id")

In [ ]:
# Parameters
simulation_id = None
slope_ci_width = 0.05
log_mean_c2c_nn_dist_width = 0.01
effective_radius_multiplier_width = 0.1

# Select where not equal to a simulation ID
data_to_query = data_with_configs
if simulation_id is not None:
    data_to_query = data_to_query.where(F.col("simulation_id") != simulation_id)

# Select within a range of slope and log_c2c_nn_dist_mean
result = (
    data_to_query
        .where((F.col("slope") > F.col("target_slope") - F.lit(slope_width))
               & (F.col("slope") <= F.col("target_slope") + F.lit(slope_width))
               & (F.col("log_mean_c2c_nn_dist") > F.col("target_log_mean_c2c_nn_dist") - F.lit(log_mean_c2c_nn_dist_width))
               & (F.col("log_mean_c2c_nn_dist") <= F.col("target_log_mean_c2c_nn_dist") + F.lit(log_mean_c2c_nn_dist_width))
               & (F.col("effective_radius_multiplier") > F.col("target_effective_radius_multiplier") - F.lit(effective_radius_multiplier_width))
               & (F.col("effective_radius_multiplier") <= F.col("target_effective_radius_multiplier") + F.lit(effective_radius_multiplier_width)))
        .groupby("target_slope", "target_log_mean_c2c_nn_dist", "target_effective_radius_multiplier")
        .agg(
            F.percentile_approx("n_craters_added_in_study_region", F.array(*[F.lit((x) / 40.0) for x in range(40)]), 10000).alias("N_percentiles"),
            F.mean("n_craters_added_in_study_region").alias("N_mean"),
            F.stddev("n_craters_added_in_study_region").alias("N_stdev"),
            F.count("n_craters_added_in_study_region").alias("count")
        )
)

In [ ]:
df = result.toPandas()
df

In [ ]:
df[df.target_slope == 2.1].sort_values(["target_log_mean_c2c_nn_dist", "target_effective_radius_multiplier"])

In [ ]:
def get_single_confidence_interval(target_slope: float,
                                   slope_delta: float,
                                   target_log_mean_c2c_nn_dist: float,
                                   log_mean_c2c_nn_dist_delta: float,
                                   target_effective_radius_multiplier: float,
                                   effective_radius_multiplier_delta: float,
                                   data: pd.DataFrame) -> Tuple[float, float, float]:
    target_slope = quantize_value(target_slope, slope_delta)
    target_log_mean_c2c_nn_dist = quantize_value(target_log_mean_c2c_nn_dist, log_mean_c2c_nn_dist_delta)
    target_effective_radius_multiplier = quantize_value(target_effective_radius_multiplier, effective_radius_multiplier_delta)
    
    row = data[(np.abs(data.target_slope - target_slope) < 0.001)
                & (np.abs(data.target_log_mean_c2c_nn_dist - target_log_mean_c2c_nn_dist) < 0.001)
                & (np.abs(data.target_effective_radius_multiplier - target_effective_radius_multiplier) < 0.001)
            ].iloc[0]

    # 2.5th and 97.5th percentiles
    return row.N_percentiles[1], row.N_percentiles[-2]

def get_confidence_intervals(target_slope: float,
                             slope_delta: float,
                             slope_ci_width: float,
                             target_log_mean_c2c_nn_dist: float,
                             log_mean_c2c_nn_dist_delta: float,
                             target_effective_radius_multiplier: float,
                             effective_radius_multiplier_delta: float,
                             data: pd.DataFrame) -> Tuple[float, float]:
    lower_ci = get_single_confidence_interval(target_slope - slope_ci_width, slope_delta, target_log_mean_c2c_nn_dist, log_mean_c2c_nn_dist_delta, target_effective_radius_multiplier, effective_radius_multiplier_delta, data)
    mean_ci = get_single_confidence_interval(target_slope, slope_delta, target_log_mean_c2c_nn_dist, log_mean_c2c_nn_dist_delta, target_effective_radius_multiplier, effective_radius_multiplier_delta, data)
    upper_ci = get_single_confidence_interval(target_slope + slope_ci_width, slope_delta, target_log_mean_c2c_nn_dist, log_mean_c2c_nn_dist_delta, target_effective_radius_multiplier, effective_radius_multiplier_delta, data)

    return [lower_ci, mean_ci, upper_ci]

In [ ]:
# Dione 1 (DEE)
# target_slope = 2.10
# slope_ci_width = 0.08
# target_log_mean_c2c_nn_dist = 0.8136322010294104

# Dione 2 (DSP)
target_slope = 2.66
slope_plus_minus = 0.05
target_log_mean_c2c_nn_dist = 0.6685316230432883
target_effective_radius_multiplier = 1.3

# Dione 3 (DICP)
# target_slope = 2.08
# slope_ci_width = 0.09
# target_log_mean_c2c_nn_dist = 0.9338405903389017

# Dione 4 (DDCP)
# target_slope = 2.15
# slope_ci_width = 0.05
# target_log_mean_c2c_nn_dist = 0.5480628421533491


get_confidence_intervals(target_slope, slope_delta, slope_ci_width, target_log_mean_c2c_nn_dist, log_mean_c2c_nn_dist_delta, target_effective_radius_multiplier, effective_radius_multiplier_delta, df)